In [ ]:
# standard libraries
from os.path import abspath
from os.path import isfile
from os.path import basename
from os.path import splitext
from os.path import abspath
from glob import glob
import re
import random
import warnings
import datetime
from operator import add
from functools import reduce

# installable with pip
import tqdm
import librosa
from scipy.io import wavfile
import numpy as np

import dawdreamer as daw

# The last cells in this notebook are optional and are for making a movie.
# You must install imagemagick and set an environment variable to magick.exe. See the MoviePy installation instructions.
# You must also install eyed3.

def get_timestamp():
    
    return datetime.datetime.now().strftime("%y-%m-%d_%H-%M-%S")

In [ ]:
# Instructions:
# You must have files named with the "Mixed In Key" Circle of Fifths convention.
# An example is "8A - 130 - My Song.wav"
# In this example, the letter A represents a "minor" key. B would have represented a major key.
# Specifically 8A refers to A Minor and 8B refers to C Major. These keys share the same
# notes, but they are distinguished by this letter.
# The number 12 could be between 1 through 12 and it represents going around the Circle of Fifths while
# staying in the same kind of key (major/minor).
# You can look up the diagram on your favorite search engine.

LYRICS_FOLDERS = ["E:/Ableton/Samples/Beatmatched Vocals-Lyrics/"]
MELODY_FOLDERS = ["E:/Ableton/Samples/Beatmatched/"]

In [ ]:
class AudioClip():
    
    def __init__(self, asd_path, wav_path, name, bpm=120, key='8', mode='A'):
        self.asd_path = asd_path
        self.wav_path = wav_path
        self.name = name
        self.bpm = bpm
        self.key = key
        self.mode = mode
        
    def __repr__(self):
        
        return f'AudioClip(bpm={self.bpm},key={self.key},mode={self.mode},name={self.name})'
        
def parse_asd_paths_to_clips(asd_paths):
    
    audioclips = []
    
    r = r"""(\d{1,2})(A|B)\s-\s(\d*)\s-\s(.*)"""
    reg = re.compile(r)

    for asd_path in asd_paths:

        wav_path = asd_path[:-4]
        if not isfile(wav_path):
            continue

        b = splitext(basename(wav_path).strip())[0]
        match = reg.search(b)

        if not match:
            print(f'Circle of Fifths regex failed for file: {b}')
            continue

        audioclip = AudioClip(asd_path, wav_path, match.group(4), bpm=int(match.group(3)), key=int(match.group(1)),
                              mode=match.group(2))        
        audioclips.append(audioclip)
    
    return audioclips


def get_pairs(clips_a, clips_b, num_desired, dist_threshold=3.5, MAX_ATTEMPTS=100000, with_replacement=True):
    
    def dist_func(clip_a, clip_b):

        dist = np.zeros((3,1))

        # compare 7B to 12B for example (5 away). Note that 1 is 1 away from 12.
        dist[0] += min(abs(clip_a.key-clip_b.key), 12-abs(clip_a.key-clip_b.key))
        
        if clip_a.mode != clip_b.mode:
            dist[1] += 0.5

        # give distance according to bpm difference
        avg_bpm = (clip_a.bpm + clip_b.bpm)/2.
        dist[2] += (abs(clip_a.bpm - clip_b.bpm)/avg_bpm)/.05

        return np.linalg.norm(dist)

    pairs = []

    attempts = 0
    num_added = 0
    
    while clips_a and clips_b and num_added < num_desired and attempts < MAX_ATTEMPTS:
        attempts += 1

        clip_a = random.choice(clips_a)
        clip_b = random.choice(clips_b)
        dist = dist_func(clip_a, clip_b)
        if dist < dist_threshold:
            pairs.append([clip_a, clip_b])
            num_added += 1
            
            if not with_replacement:
                clips_a.remove(clip_a)
                clips_b.remove(clip_b)
            
    return pairs

In [ ]:
vocal_asd_paths = reduce(add,[list(glob(folder+'*.asd')) for folder in LYRICS_FOLDERS])
melody_asd_paths = reduce(add,[list(glob(folder+'*.asd')) for folder in MELODY_FOLDERS])

vocal_asd_paths = [x.replace('\\', '/') for x in vocal_asd_paths]
melody_asd_paths = [x.replace('\\', '/') for x in melody_asd_paths]

vocal_clips = parse_asd_paths_to_clips(vocal_asd_paths)
melody_clips = parse_asd_paths_to_clips(melody_asd_paths)

num_desired = 5
dist_threshold=3.3
with_replacement = False

pairs = get_pairs(vocal_clips, melody_clips, num_desired, dist_threshold=dist_threshold, with_replacement=with_replacement)
# print('VOCAL' + ' '*46 + '| MELODY')
# for vocal_clip, melody_clip in pairs:
#     print(vocal_clip.name[:50] + (' '*(50-len(vocal_clip.name)))+ ' | ' + melody_clip.name)

In [ ]:
SAMPLE_RATE = 44100
BLOCK_SIZE = 512

def load_audio_file(file_path, duration=None):
    
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")

    #import soundfile
    #sig, rate = soundfile.read(file_path, always_2d=True, samplerate=SAMPLE_RATE, stop=int(duration*SAMPLE_RATE))	

    sig, rate = librosa.load(file_path, duration=duration, mono=False, sr=SAMPLE_RATE)
    assert(rate == SAMPLE_RATE)
    
    if sig.ndim == 1:
        sig = np.stack([sig,sig])
    
    return sig

def render(engine, file_path=None, duration=5.):

    engine.render(duration)

    output = engine.get_audio()

    if file_path is not None:

        wavfile.write(file_path, SAMPLE_RATE, output.transpose())

    return True

In [ ]:
engine = daw.RenderEngine(SAMPLE_RATE, BLOCK_SIZE)

vocal_processor = engine.make_playbackwarp_processor("vocals", np.zeros((2, 1)))
melody_processor = engine.make_playbackwarp_processor("melody", np.zeros((2, 1)))

graph = [
    (vocal_processor, []),
    (melody_processor, []),
    (engine.make_add_processor("add", [.7, .7]), ["vocals", "melody"])
]

assert(engine.load_graph(graph))

all_audio = np.zeros((2, 0))

durations = []

for vocal_clip, melody_clip in tqdm.tqdm(pairs, desc="Rendering audio"):

    # average the bpms
    bpm = (vocal_clip.bpm+melody_clip.bpm)/2.
    engine.set_bpm(bpm)
    
    num_quarter_notes = 16

    duration = num_quarter_notes * (60./bpm)

    print('melody_clip.wav_path: ', melody_clip.wav_path)
    data = load_audio_file(melody_clip.wav_path)
    print('set data')
    melody_processor.set_data(data)
    print('vocal_clip.wav_path: ', vocal_clip.wav_path)
    data = load_audio_file(vocal_clip.wav_path)
    print('set data')
    vocal_processor.set_data(data)

    print('melody_processor.set_clip_file')
    if not melody_processor.set_clip_file(melody_clip.asd_path):
        durations.append(0.)
        continue

    print('vocal_processor.set_clip_file')
    if not vocal_processor.set_clip_file(vocal_clip.asd_path):
        durations.append(0.)
        continue

    durations.append(duration)

#     print('melody_processor.start_marker: ', melody_processor.start_marker)
#     print('melody_processor.end_marker: ', melody_processor.end_marker)
#     print('melody_processor.loop_on: ', melody_processor.loop_on)
#     print('melody_processor.loop_start: ', melody_processor.loop_start)
#     print('melody_processor.loop_end: ', melody_processor.loop_end)
#     print('melody_processor.warp_on: ', melody_processor.warp_on)

#     print('vocal_processor.start_marker: ', vocal_processor.start_marker)
#     print('vocal_processor.end_marker: ', vocal_processor.end_marker)
#     print('vocal_processor.loop_on: ', vocal_processor.loop_on)
#     print('vocal_processor.loop_start: ', vocal_processor.loop_start)
#     print('vocal_processor.loop_end: ', vocal_processor.loop_end)
#     print('vocal_processor.warp_on: ', vocal_processor.warp_on)

    print('render')
    engine.render(duration)

    all_audio = np.concatenate([all_audio, engine.get_audio()], axis=-1)

# convert from float 32 to int16
all_audio_out = (np.iinfo(np.int16).max * (all_audio / np.max(all_audio))).astype(np.int16)
wavfile.write(f'output/all_audio_{get_timestamp()}.wav', SAMPLE_RATE, all_audio_out.transpose())

In [ ]:
# This optional section is for making a movie.
# You must install imagemagick and set an environment variable to magick.exe. See the MoviePy installation instructions.
# You must also install eyed3.
from moviepy.editor import *
from moviepy.audio.AudioClip import AudioArrayClip
import eyed3

screensize = (1280,720)

def audioclip_to_title(audioclip):
    
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            audio_meta = eyed3.load(audioclip.wav_path)
            if audio_meta:
                return f'{audioclip.key}{audioclip.mode} - {audioclip.bpm}\n{audio_meta.tag.artist}\n{audio_meta.tag.title}'
            else:
                print(f'ID3 not recognized for file: {audioclip.wav_path}')
    except IOError as e:
        print(f'Unrecognized file: {audioclip.wav_path}')
    except Exception as e:
        print(f'Error loading ID3 tags from file: {audioclip.wav_path}')
    
    return audioclip.name

total_duration = 0.
video_clips = []
for (vocal_clip, melody_clip), duration in zip(pairs, durations):
    
    bpm = (vocal_clip.bpm+melody_clip.bpm)/2.
    engine.set_bpm(bpm)
    
    if duration == 0.:
        continue
    total_duration += duration
    
    txt = f'{audioclip_to_title(vocal_clip)}\n\n&\n\n{audioclip_to_title(melody_clip)}'
    txtClip = TextClip(txt, color='white', font="Amiri-Bold", kerning=2, fontsize=36, size=screensize).set_duration(duration)
    
    video_clips.append(txtClip)

final_clip = concatenate_videoclips(video_clips)
final_clip.audio = AudioArrayClip(all_audio.transpose(), fps=SAMPLE_RATE)
final_clip.write_videofile(f'output/my_movie_{get_timestamp()}.mp4',fps=4,codec='mpeg4')
    
# showing video 
final_clip.ipython_display(width=720, fps=10, maxduration=60*15.)

In [ ]:
filepath = "E:/Ableton/Samples/Beatmatched Vocals-Lyrics/2A - 80 - Who Run It (Acappella).mp3"
data = load_audio_file(filepath)
print(data.shape)
engine = daw.RenderEngine(SAMPLE_RATE, BLOCK_SIZE)
vocal_processor = engine.make_playbackwarp_processor("vocals", data)
assert(vocal_processor.set_clip_file(filepath+'.asd'))
vocal_processor.warp_markers

In [ ]:
import os
os.remove('__temp__.mp4')